In [5]:
#camera access for laptop 

import cv2

def turn_on_camera():
    # Turn on the system camera (open the video capture)
    cap = cv2.VideoCapture(0)  # '0' is the default webcam, change it if you have multiple cameras

    if not cap.isOpened():
        print("Error: Camera could not be opened.")
        return None
    
    print("Camera is now ON")

    while True:
        ret, frame = cap.read()

        if not ret:
            print("Failed to grab frame")
            break

        # Display the camera feed
        cv2.imshow('Camera Feed', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Turning off the camera...")
            break

    # Turn off the camera (release the capture)
    cap.release()
    cv2.destroyAllWindows()

def turn_off_camera():
    # You can simulate turning off by releasing the camera capture object
    print("Camera is now OFF")

# Usage
turn_on_camera()  # This will turn on the camera and display the feed

# You can call `turn_off_camera()` after closing the feed with 'q', but it's not necessary


Camera is now ON
Turning off the camera...


In [6]:
#to capture Image

import cv2

def capture_image():
    # Open the default camera (camera 0)
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Camera could not be opened.")
        return

    print("Press 'c' or spacebar to capture an image. Press 'q' to quit.")

    while True:
        # Capture frame-by-frame from the camera feed
        ret, frame = cap.read()

        if not ret:
            print("Failed to grab frame")
            break

        # Display the resulting frame
        cv2.imshow('Live Camera', frame)

        # Wait for the user's keypress
        key = cv2.waitKey(1) & 0xFF

        # If 'c' or spacebar is pressed, capture the image
        if key == ord('c') or key == 32:  # 32 is the ASCII code for spacebar
            img_name = "captured_image.png"
            cv2.imwrite(img_name, frame)  # Save the image as a PNG file
            print(f"Image captured and saved as {img_name}")

        # If 'q' is pressed, quit the loop
        elif key == ord('q'):
            print("Exiting...")
            break

    # Release the camera and close the window
    cap.release()
    cv2.destroyAllWindows()

# Call the function to capture an image
capture_image()


Press 'c' or spacebar to capture an image. Press 'q' to quit.
Image captured and saved as captured_image.png
Image captured and saved as captured_image.png
Image captured and saved as captured_image.png
Image captured and saved as captured_image.png
Image captured and saved as captured_image.png
Image captured and saved as captured_image.png
Image captured and saved as captured_image.png
Exiting...


In [9]:
#camera access and capturing the image mulitple times 

import cv2
import time

# Initialize the video capture (0 is usually the default camera)
cap = cv2.VideoCapture(0)

img_counter = 0  # Counter to create unique image file names

while True:
    # Capture the video frame-by-frame
    ret, frame = cap.read()

    # Display the resulting frame
    cv2.imshow('Press Space or c to capture image', frame)

    # Wait for a key press (1ms delay between frames)
    key = cv2.waitKey(1) & 0xFF

    # If 'c' or spacebar is pressed, capture the image
    if key == ord('c') or key == 32:  # 32 is the ASCII code for spacebar
        img_name = f"captured_image_{img_counter}.png"
        cv2.imwrite(img_name, frame)  # Save the image as a PNG file
        print(f"Image captured and saved as {img_name}")
        img_counter += 1  # Increment the counter for unique file names

    # If 'q' is pressed, exit the loop
    if key == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()
